In [1]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder , StandardScaler , MinMaxScaler , RobustScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from collections import Counter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, classification_report
from sklearn.metrics import recall_score




### Read Data

In [2]:

diabetes = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")
diabetes.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


### Remove Outliers

In [3]:
def drop_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_cleaned = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_cleaned

diabetes_cleaned = drop_outliers_iqr(diabetes, 'BMI')
diabetes_cleaned = drop_outliers_iqr(diabetes_cleaned, 'GenHlth')

### Split Data

In [6]:

x_train, x_test = train_test_split(diabetes_cleaned, test_size=0.8, random_state=42)
x_test,y_test = x_test.drop(['Diabetes_binary'],axis=1),x_test['Diabetes_binary']
x_train,y_train = x_train.drop(['Diabetes_binary'],axis=1),x_train['Diabetes_binary']


### Standardize Data

In [7]:
# scaler = StandardScaler()
# scaler = RobustScaler()
scaler = StandardScaler()

scaler.fit(x_train)
x_train_scaler = scaler.transform(x_train)
x_test_scaler = scaler.transform(x_test)

### TEST 1 USE Smothing

In [9]:
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train_scaler, y_train)

model = SVC(
    C=0.1,
    random_state=42,
    kernel='rbf',
    gamma='scale',
    degree=2,
    class_weight='balanced',
    probability=True
)

model.fit(x_train_resampled, y_train_resampled)

y_train_pred = model.predict(x_train_resampled)
y_test_pred = model.predict(x_test_scaler)

train_accuracy = accuracy_score(y_train_resampled, y_train_pred) * 100
test_accuracy = accuracy_score(y_test, y_test_pred) * 100

print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(classification_report(y_test, y_test_pred, target_names=['No Diabetes', 'Diabetes']))

def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['No Diabetes', 'Diabetes'],
                yticklabels=['No Diabetes', 'Diabetes'])
    plt.title(title)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

# plot_confusion_matrix(y_train_resampled, y_train_pred, "Training Set Confusion Matrix")
# plot_confusion_matrix(y_test, y_test_pred, "Test Set Confusion Matrix")


Training Accuracy: 78.14%
Test Accuracy: 71.43%
              precision    recall  f1-score   support

 No Diabetes       0.96      0.71      0.81    163869
    Diabetes       0.26      0.77      0.39     22499

    accuracy                           0.71    186368
   macro avg       0.61      0.74      0.60    186368
weighted avg       0.87      0.71      0.76    186368



### Search For Best Parameters Wit Cross-Validation 

In [ ]:
recall_class1_scorer = make_scorer(
    recall_score, 
    pos_label=1, 
    average='binary' 
)

svm = SVC(random_state=42, probability=True)

param_grid = [
    {
        'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
        'kernel': ['linear', 'rbf', 'poly'], 
        'class_weight': ['balanced', {0: 1, 1: 6}],  
        'gamma': ['scale', 'auto'] 
    }
]

grid_search = GridSearchCV(
    svm, 
    param_grid, 
    cv=5,  
    scoring=recall_class1_scorer, 
    n_jobs=-1,
    verbose=1
)

grid_search.fit(x_train_scaler, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(x_test_scaler)

print("Best Parameters:", grid_search.best_params_)
print("\nTest Accuracy: {:.2%}".format(accuracy_score(y_test, y_pred)))


### TEST 2 NOT USE Smothing

In [ ]:
svm =  SVC(
    C=0.1,
    random_state=42,
    kernel='rbf',
    gamma='scale',
    degree=2,
    class_weight='balanced',
    probability=True
)

svm.fit(x_train_scaler, y_train)

y_train_pred = svm.predict(x_train_scaler)
y_test_pred = svm.predict(x_test_scaler)

train_accuracy = accuracy_score(y_train, y_train_pred) * 100
test_accuracy = accuracy_score(y_test, y_test_pred) * 100

print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")

def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['No Diabetes', 'Diabetes'],
                yticklabels=['No Diabetes', 'Diabetes'])
    plt.title(title)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

# plot_confusion_matrix(y_train, y_train_pred, "Training Set Confusion Matrix")
# plot_confusion_matrix(y_test, y_test_pred, "Test Set Confusion Matrix")

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_test_pred,target_names=['No Diabetes','Diabetes']))


### Test 3 Change Threshold

In [ ]:
y_probs = svm_model.predict_proba(x_test_scaler)[:, 1]
y_train_probs = svm_model.predict_proba(x_train_scaler)[:, 1]

threshold = 0.4
y_train_pred = (y_train_probs >= threshold).astype(int)
y_test_pred = (y_probs >= threshold).astype(int)

train_accuracy = accuracy_score(y_train, y_train_pred) * 100
test_accuracy = accuracy_score(y_test, y_test_pred) * 100

print(f"Training Accuracy with Threshold {threshold}: {train_accuracy:.2f}%")
print(f"Test Accuracy with Threshold {threshold}: {test_accuracy:.2f}%")

print("\nClassification Report (Test Set) with Threshold =", threshold)
print(classification_report(y_test, y_test_pred, target_names=['No Diabetes', 'Diabetes']))

### Test 4 Get the top 10 Features

In [ ]:

selector = SelectKBest(f_classif, k=10)
X_new = selector.fit_transform(x_train, y_train)

selected_features = x_train.columns[selector.get_support()]
print("Best features:", selected_features)

In [ ]:

X_train_selected = selector.transform(x_train)
X_test_selected = selector.transform(x_test)
scaler = StandardScaler()

scaler.fit(X_train_selected)
x_train_scaler = scaler.transform(X_train_selected)
x_test_scaler = scaler.transform(X_test_selected)

svm_model = 

svm_model.fit(x_train_scaler, y_train)

y_train_pred = svm_model.predict(x_train_scaler)
y_test_pred = svm_model.predict(x_test_scaler)

train_accuracy = accuracy_score(y_train, y_train_pred) * 100
test_accuracy = accuracy_score(y_test, y_test_pred) * 100

print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(classification_report(y_test, y_test_pred, target_names=['No Diabetes', 'Diabetes']))
